In [29]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 25: Combo Breaker ---</h2><p>You finally reach the check-in desk. Unfortunately, their registration systems are currently offline, and they cannot check you in. Noticing the look on your face, they quickly add that tech support is already on the way! They even created all the room keys this morning; you can take yours now and give them your room deposit once the registration system comes back online.</p>
<p>The room key is a small <a href="https://en.wikipedia.org/wiki/Radio-frequency_identification" target="_blank">RFID</a> card. Your room is on the 25th floor and the elevators are also temporarily out of service, so it takes what little energy you have left to even climb the stairs and navigate the halls. You finally reach the door to your room, swipe your card, and - <em>beep</em> - the light turns red.</p>
<p>Examining the card more closely, you discover a phone number for tech support.</p>
<p>"Hello! How can we help you today?" You explain the situation.</p>
<p>"Well, it sounds like the card isn't sending the right command to unlock the door. If you go back to the check-in desk, surely someone there can reset it for you." Still catching your breath, you describe the status of the elevator and the exact number of stairs you just had to climb.</p>
<p>"I see! Well, your only other option would be to reverse-engineer the cryptographic handshake the card does with the door and then inject your own commands into the data stream, but that's definitely impossible." You thank them for their time.</p>
<p>Unfortunately for the door, you know a thing or two about cryptographic handshakes.</p>
<p>The handshake used by the card and the door involves an operation that <em>transforms</em> a <em>subject number</em>. To transform a subject number, start with the value <code>1</code>. Then, a number of times called the <em>loop size</em>, perform the following steps:</p>
<ul>
<li>Set the value to itself multiplied by the <em>subject number</em>.</li>
<li>Set the value to the remainder after dividing the value by <em><code>20201227</code></em>.</li>
</ul>
<p>The card always uses a specific, secret <em>loop size</em> when it transforms a subject number. The door always uses a different, secret loop size.</p>
<p>The cryptographic handshake works like this:</p>
<ul>
<li>The <em>card</em> transforms the subject number of <em><code>7</code></em> according to the <em>card's</em> secret loop size. The result is called the <em>card's public key</em>.</li>
<li>The <em>door</em> transforms the subject number of <em><code>7</code></em> according to the <em>door's</em> secret loop size. The result is called the <em>door's public key</em>.</li>
<li>The card and door use the wireless RFID signal to transmit the two public keys (your puzzle input) to the other device. Now, the <em>card</em> has the <em>door's</em> public key, and the <em>door</em> has the <em>card's</em> public key. Because you can eavesdrop on the signal, you have both public keys, but neither device's loop size.</li>
<li>The <em>card</em> transforms the subject number of <em>the door's public key</em> according to the <em>card's</em> loop size. The result is the <em>encryption key</em>.</li>
<li>The <em>door</em> transforms the subject number of <em>the card's public key</em> according to the <em>door's</em> loop size. The result is the same <em>encryption key</em> as the <em>card</em> calculated.</li>
</ul>
<p>If you can use the two public keys to determine each device's loop size, you will have enough information to calculate the secret <em>encryption key</em> that the card and door use to communicate; this would let you send the <code>unlock</code> command directly to the door!</p>
<p>For example, suppose you know that the card's public key is <code>5764801</code>. With a little trial and error, you can work out that the card's loop size must be <em><code>8</code></em>, because transforming the initial subject number of <code>7</code> with a loop size of <code>8</code> produces <code>5764801</code>.</p>
<p>Then, suppose you know that the door's public key is <code>17807724</code>. By the same process, you can determine that the door's loop size is <em><code>11</code></em>, because transforming the initial subject number of <code>7</code> with a loop size of <code>11</code> produces <code>17807724</code>.</p>
<p>At this point, you can use either device's loop size with the other device's public key to calculate the <em>encryption key</em>. Transforming the subject number of <code>17807724</code> (the door's public key) with a loop size of <code>8</code> (the card's loop size) produces the encryption key, <em><code>14897079</code></em>. (Transforming the subject number of <code>5764801</code> (the card's public key) with a loop size of <code>11</code> (the door's loop size) produces the same encryption key: <em><code>14897079</code></em>.)</p>
<p><em>What encryption key is the handshake trying to establish?</em></p>
</article>


In [30]:
def transform(subject_number: int, loop_size: int) -> int:
    value = 1
    for _ in range(loop_size):
        value *= subject_number
        value %= 20201227
    return value


def get_loop_size(public_key: int, subject_number: int) -> int:
    value, loop_size = 1, 0
    while value != public_key:
        value *= subject_number
        value %= 20201227
        loop_size += 1
    return loop_size


assert get_loop_size(5764801, 7) == 8
assert get_loop_size(17807724, 7) == 11


def cryptographic_handshake(public_key_card: int, public_key_door: int) -> int:
    loopsize_card = get_loop_size(public_key_card, 7)
    loopsize_door = get_loop_size(public_key_door, 7)

    encryption_key_1 = transform(public_key_door, loopsize_card)
    encryption_key_2 = transform(public_key_card, loopsize_door)

    assert encryption_key_1 == encryption_key_2
    return encryption_key_1


assert cryptographic_handshake(5764801, 17807724) == 14897079

In [31]:
print(f"Part I: {cryptographic_handshake(15628416, 11161639)}")

Part I: 19774660


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>19774660</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p><span title="You notice the brand written on the side of the lock: Diffie, Hellman, and Merkle.">The light turns green and the door unlocks.</span> As you collapse onto the bed in your room, your pager goes off!</p>
<p>"It's an emergency!" the Elf calling you explains. "The <a href="https://en.wikipedia.org/wiki/Soft_serve" target="_blank">soft serve</a> machine in the cafeteria on sub-basement 7 just failed and you're the only one that knows how to fix it! We've already dispatched a reindeer to your location to pick you up."</p>
<p>You hear the sound of hooves landing on your balcony.</p>
<p>The reindeer carefully explores the contents of your room while you figure out how you're going to pay the <em class="star">50 stars</em> you owe the resort before you leave. Noticing that you look concerned, the reindeer wanders over to you; you see that it's carrying a small pouch.</p>
<p>"Sorry for the trouble," a note in the pouch reads. Sitting at the bottom of the pouch is a gold coin with a little picture of a starfish on it.</p>
<p>Looks like you only needed <em class="star">49 stars</em> after all.</p>
</article>

</main>


<link href="style.css" rel="stylesheet"></link>

<main>
<article>
<p>You spend all <span class="day-success">fifty stars</span> to cover the room deposit!</p><p>As you fix the soft serve machine, Santa offers you a ride in his sleigh; maybe the resort has a chimney you can use...</p>
<p class="aside">Congratulations!  You've finished every puzzle in Advent of Code 2020!  I hope you had as much fun solving them as I had making them for you.  I'd love to hear about your adventure; you can get in touch with me via contact info on <a href="http://was.tl/" target="_blank">my website</a> or through <a href="https://twitter.com/ericwastl" target="_blank">Twitter</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">If you'd like to see more things like this in the future, please consider <a href="/2020/support" target="_blank">supporting</a> Advent of Code and sharing it with others.</p>
<p class="aside">To hear about future projects, you can follow me on <a href="https://twitter.com/ericwastl" target="_blank">Twitter</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">I've <span style="border-bottom:1px dotted #ffff66;" title="Yep, just like that.  There's at least one in the description for each day.">highlighted</span> the easter eggs in each puzzle, just in case you missed any.  Hover your mouse over them, and the easter egg will appear.</p>
<p>You can <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I+just+completed+all+25+days+of+Advent+of+Code+2020%21&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I+just+completed+all+25+days+of+Advent+of+Code+2020%21+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this moment with your friends, or <a href="/2020">[Go Check on Your Calendar]</a>.
</p></article>
<style>
.calendar-bkg {
  background: linear-gradient(to bottom, rgba(255,255,255,0) 0%,rgba(255,255,255,.3) 100%);
}
.sf {
  position: fixed;
  animation: anim-sf0 linear 20s infinite;
  z-index: -1;
}
.sf:before {
  content: "*";
}
.sf.sf0 { color: #ffffff; }
.sf.sf1 { color: #cccccc; animation-name: anim-sf1; }
.sf.sf2 { color: #999999; animation-name: anim-sf2; }
.sf.sf3 { color: #666666; animation-name: anim-sf3; }
@keyframes anim-sf0 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(8em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(8em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf1 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(6em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(6em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf2 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(4em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(4em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf3 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(2em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(2em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
</style>
<div class="sf sf0" style="top:-1.86em; left:44.07vw; animation-delay:-5.64s;"></div><div class="sf sf3" style="top:-4.73em; left:82.03vw; animation-delay:-10.85s;"></div><div class="sf sf3" style="top:-4.96em; left:46.48vw; animation-delay:-8.45s;"></div><div class="sf sf3" style="top:-4.16em; left:87.64vw; animation-delay:-13.98s;"></div><div class="sf sf0" style="top:-1.45em; left:93.27vw; animation-delay:-10.32s;"></div><div class="sf sf0" style="top:-1.85em; left:17.80vw; animation-delay:-7.27s;"></div><div class="sf sf0" style="top:-1.32em; left:84.61vw; animation-delay:-4.83s;"></div><div class="sf sf3" style="top:-4.53em; left:92.09vw; animation-delay:-15.66s;"></div><div class="sf sf2" style="top:-3.26em; left:65.54vw; animation-delay:-4.46s;"></div><div class="sf sf0" style="top:-1.50em; left:1.69vw; animation-delay:-14.78s;"></div><div class="sf sf2" style="top:-3.76em; left:3.39vw; animation-delay:-9.67s;"></div><div class="sf sf2" style="top:-3.14em; left:93.61vw; animation-delay:-16.74s;"></div><div class="sf sf1" style="top:-2.46em; left:85.82vw; animation-delay:-18.12s;"></div><div class="sf sf2" style="top:-3.20em; left:53.17vw; animation-delay:-2.28s;"></div><div class="sf sf0" style="top:-1.30em; left:91.10vw; animation-delay:-1.36s;"></div><div class="sf sf2" style="top:-3.70em; left:58.37vw; animation-delay:-10.25s;"></div><div class="sf sf3" style="top:-4.68em; left:9.28vw; animation-delay:-12.21s;"></div><div class="sf sf2" style="top:-3.53em; left:46.61vw; animation-delay:-10.90s;"></div><div class="sf sf3" style="top:-4.21em; left:62.96vw; animation-delay:-6.55s;"></div><div class="sf sf0" style="top:-1.38em; left:17.07vw; animation-delay:-0.59s;"></div><div class="sf sf0" style="top:-1.71em; left:80.59vw; animation-delay:-10.11s;"></div><div class="sf sf3" style="top:-4.43em; left:70.63vw; animation-delay:-2.89s;"></div><div class="sf sf3" style="top:-4.46em; left:17.84vw; animation-delay:-13.05s;"></div><div class="sf sf2" style="top:-3.14em; left:74.88vw; animation-delay:-2.84s;"></div><div class="sf sf2" style="top:-3.68em; left:78.46vw; animation-delay:-12.68s;"></div><div class="sf sf0" style="top:-1.28em; left:16.97vw; animation-delay:-0.03s;"></div><div class="sf sf3" style="top:-4.64em; left:46.08vw; animation-delay:-8.52s;"></div><div class="sf sf1" style="top:-2.35em; left:45.10vw; animation-delay:-19.41s;"></div><div class="sf sf2" style="top:-3.11em; left:61.76vw; animation-delay:-2.62s;"></div><div class="sf sf1" style="top:-2.34em; left:72.13vw; animation-delay:-8.37s;"></div><div class="sf sf2" style="top:-3.16em; left:48.76vw; animation-delay:-13.06s;"></div><div class="sf sf0" style="top:-1.78em; left:59.32vw; animation-delay:-13.51s;"></div><div class="sf sf2" style="top:-3.50em; left:61.92vw; animation-delay:-8.64s;"></div><div class="sf sf3" style="top:-4.11em; left:8.92vw; animation-delay:-11.44s;"></div><div class="sf sf2" style="top:-3.26em; left:22.87vw; animation-delay:-18.95s;"></div><div class="sf sf3" style="top:-4.30em; left:15.44vw; animation-delay:-3.76s;"></div><div class="sf sf2" style="top:-3.19em; left:80.22vw; animation-delay:-5.34s;"></div><div class="sf sf1" style="top:-2.80em; left:43.51vw; animation-delay:-18.77s;"></div><div class="sf sf1" style="top:-2.03em; left:45.06vw; animation-delay:-15.80s;"></div><div class="sf sf0" style="top:-1.86em; left:23.69vw; animation-delay:-9.57s;"></div><div class="sf sf0" style="top:-1.87em; left:62.22vw; animation-delay:-9.36s;"></div><div class="sf sf1" style="top:-2.68em; left:62.60vw; animation-delay:-1.66s;"></div><div class="sf sf0" style="top:-1.40em; left:65.86vw; animation-delay:-4.18s;"></div><div class="sf sf0" style="top:-1.47em; left:44.53vw; animation-delay:-1.63s;"></div><div class="sf sf1" style="top:-2.77em; left:20.61vw; animation-delay:-17.61s;"></div><div class="sf sf3" style="top:-4.32em; left:33.93vw; animation-delay:-13.25s;"></div><div class="sf sf3" style="top:-4.09em; left:29.70vw; animation-delay:-6.08s;"></div><div class="sf sf0" style="top:-1.40em; left:71.02vw; animation-delay:-16.96s;"></div><div class="sf sf0" style="top:-1.60em; left:26.03vw; animation-delay:-18.21s;"></div><div class="sf sf3" style="top:-4.76em; left:67.21vw; animation-delay:-14.70s;"></div><div class="sf sf3" style="top:-4.61em; left:5.04vw; animation-delay:-5.26s;"></div><div class="sf sf2" style="top:-3.79em; left:47.19vw; animation-delay:-3.04s;"></div><div class="sf sf3" style="top:-4.62em; left:94.63vw; animation-delay:-9.85s;"></div><div class="sf sf2" style="top:-3.23em; left:41.09vw; animation-delay:-14.63s;"></div><div class="sf sf3" style="top:-4.07em; left:32.44vw; animation-delay:-15.84s;"></div><div class="sf sf3" style="top:-4.34em; left:67.34vw; animation-delay:-0.46s;"></div><div class="sf sf3" style="top:-4.53em; left:24.63vw; animation-delay:-10.34s;"></div><div class="sf sf3" style="top:-4.96em; left:65.85vw; animation-delay:-13.54s;"></div><div class="sf sf2" style="top:-3.31em; left:87.33vw; animation-delay:-2.17s;"></div><div class="sf sf1" style="top:-2.74em; left:95.35vw; animation-delay:-18.43s;"></div><div class="sf sf1" style="top:-2.86em; left:17.96vw; animation-delay:-19.97s;"></div><div class="sf sf1" style="top:-2.09em; left:34.94vw; animation-delay:-10.35s;"></div><div class="sf sf3" style="top:-4.48em; left:8.00vw; animation-delay:-18.38s;"></div><div class="sf sf3" style="top:-4.10em; left:81.48vw; animation-delay:-2.80s;"></div><div class="sf sf2" style="top:-3.36em; left:1.80vw; animation-delay:-10.42s;"></div><div class="sf sf2" style="top:-3.88em; left:57.46vw; animation-delay:-12.96s;"></div><div class="sf sf2" style="top:-3.38em; left:22.82vw; animation-delay:-0.63s;"></div><div class="sf sf3" style="top:-4.59em; left:48.23vw; animation-delay:-0.87s;"></div><div class="sf sf3" style="top:-4.33em; left:53.57vw; animation-delay:-14.93s;"></div><div class="sf sf0" style="top:-1.67em; left:72.31vw; animation-delay:-19.70s;"></div><div class="sf sf3" style="top:-4.86em; left:64.85vw; animation-delay:-10.32s;"></div><div class="sf sf1" style="top:-2.81em; left:25.13vw; animation-delay:-12.53s;"></div><div class="sf sf0" style="top:-1.83em; left:81.76vw; animation-delay:-0.45s;"></div><div class="sf sf0" style="top:-1.17em; left:8.48vw; animation-delay:-3.74s;"></div><div class="sf sf1" style="top:-2.26em; left:75.41vw; animation-delay:-11.56s;"></div><div class="sf sf0" style="top:-1.82em; left:21.68vw; animation-delay:-2.17s;"></div><div class="sf sf1" style="top:-2.60em; left:73.61vw; animation-delay:-7.36s;"></div><div class="sf sf2" style="top:-3.20em; left:74.90vw; animation-delay:-17.39s;"></div><div class="sf sf3" style="top:-4.77em; left:7.08vw; animation-delay:-10.97s;"></div><div class="sf sf1" style="top:-2.86em; left:60.73vw; animation-delay:-10.76s;"></div><div class="sf sf1" style="top:-2.66em; left:59.51vw; animation-delay:-7.69s;"></div><div class="sf sf1" style="top:-2.94em; left:26.62vw; animation-delay:-18.13s;"></div><div class="sf sf1" style="top:-2.95em; left:65.34vw; animation-delay:-13.05s;"></div><div class="sf sf2" style="top:-3.98em; left:34.00vw; animation-delay:-12.71s;"></div><div class="sf sf1" style="top:-2.15em; left:12.65vw; animation-delay:-17.25s;"></div><div class="sf sf3" style="top:-4.17em; left:78.30vw; animation-delay:-3.49s;"></div><div class="sf sf0" style="top:-1.03em; left:41.86vw; animation-delay:-9.40s;"></div><div class="sf sf2" style="top:-3.30em; left:19.61vw; animation-delay:-6.71s;"></div><div class="sf sf2" style="top:-3.96em; left:21.27vw; animation-delay:-2.65s;"></div><div class="sf sf1" style="top:-2.19em; left:79.36vw; animation-delay:-6.64s;"></div><div class="sf sf1" style="top:-2.58em; left:35.99vw; animation-delay:-4.89s;"></div><div class="sf sf1" style="top:-2.61em; left:15.78vw; animation-delay:-15.22s;"></div><div class="sf sf1" style="top:-2.03em; left:49.58vw; animation-delay:-8.72s;"></div><div class="sf sf1" style="top:-2.37em; left:49.26vw; animation-delay:-9.63s;"></div><div class="sf sf2" style="top:-3.69em; left:40.06vw; animation-delay:-9.04s;"></div><div class="sf sf2" style="top:-3.41em; left:62.98vw; animation-delay:-12.05s;"></div><div class="sf sf1" style="top:-2.42em; left:52.23vw; animation-delay:-17.13s;"></div><div class="sf sf3" style="top:-4.54em; left:64.57vw; animation-delay:-16.27s;"></div><div class="sf sf0" style="top:-1.86em; left:67.60vw; animation-delay:-5.78s;"></div><div class="sf sf0" style="top:-1.72em; left:41.34vw; animation-delay:-2.26s;"></div>
</main>
